Spark LDA for NASA ADS v1.0 
8/2/2016
@author: Tarun Ruchandani

To Do:
x Generate document term matrix for LDA
x Get LDA Results on small Corpus

8/3:

o Generate a JS Matrix per mapequation's input
- Generate topic trend reports.
o Build Mapequation with JS Matrix
o Extend to a larger corpus
o Build Docker Container
o Deploy to adsqb

- JS Matrix: figure out the right implementation - also try ifa library


In [12]:
#All 'em libraries

import numpy as np
import textmining
import pyspark
import lda
import lda.datasets
import ads
import pandas as pd
from textblob import TextBlob
from numpy  import array
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [17]:
# Extract Cosmology papers
ads.config.token = 'TkEI7jQxScyoCtzHfhcpCWVelRqySmH5XBksQCFA'
papers = list(ads.SearchQuery(q='cosmology',start=100,rows=100))

paper_abstracts=list()

for paper in papers:
    paper_abstracts.append(paper.abstract)

# Select first 25 papers which have abstracts. Some later ones don't. Revisit this when scaling.
# paper_abstracts = paper_abstracts[0:25]
print(len(paper_abstracts))

/Users/tarunruchandani/anaconda3/lib/python3.5/site-packages/ads/utils.py:23: UserWarning: You are lazy loading attributes via 'abstract', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,


100


In [4]:
# Build Term-Document Matrix

tdm = textmining.TermDocumentMatrix()
for paper in paper_abstracts:
    tdm.add_doc(paper)
    
paper_abstracts_tdm = list()

for row in tdm.rows(cutoff=0):
    paper_abstracts_tdm.append(row)

paper_abstracts_tdm_df = pd.DataFrame(paper_abstracts_tdm)
paper_abstracts_tdm_df.head()

# Vocab of terms in abstracts
vocab = paper_abstracts_tdm_df._slice(slice(1))
vocab

d_t_freq = paper_abstracts_tdm_df._slice(slice(1,24))

b=d_t_freq.values

b=b.astype(int)


# d_t_freq_m = d_t_freq.as_matrix
# d_t_freq_np = np.ndarray(b, dtype=np.int64)
# d_t_freq_np


In [5]:
# running LDA on TDM
# Parameters:	
# rdd – RDD of documents, which are tuples of document IDs and term (word) count vectors. 
# The term count vectors are “bags of words” with a fixed-size vocabulary 
# (where the vocabulary size is the length of the vector). Document IDs must be unique and >= 0.

# k – Number of topics to infer, i.e., the number of soft cluster centers. (default: 10)

# maxIterations – Maximum number of iterations allowed. (default: 20)

# docConcentration – Concentration parameter (commonly named “alpha”) 
# for the prior placed on documents’ distributions over topics (“theta”). (default: -1.0)

# topicConcentration – Concentration parameter (commonly named “beta” or “eta”) 
# for the prior placed on topics’ distributions over terms. (default: -1.0)

# seed – Random seed for cluster initialization. Set as None to generate seed based on system time. (default: None)

# checkpointInterval – Period (in iterations) between checkpoints. (default: 10)

# optimizer – LDAOptimizer used to perform the actual calculation. Currently “em”, “online” are supported. 
# (default: “em”)

model = lda.LDA(n_topics=20, n_iter=500, random_state=1)
model.fit(b)

topic_word = model.topic_word_  # model.components_ also works


for i, topic_dist in enumerate(topic_word):
     topic_words = np.array(vocab)


In [6]:
doc_topic = model.doc_topic_
print("type(doc_topic): {}".format(type(doc_topic)))
print("shape: {}".format(doc_topic.shape))

type(doc_topic): <class 'numpy.ndarray'>
shape: (23, 20)


In [7]:
# Document-topic Probab

doc_topic = model.doc_topic_

for n in range(5):
    sum_pr = sum(doc_topic[n,:])
    print("document: {} sum: {}".format(n, sum_pr))

for n in range(10):
    topic_most_pr = doc_topic[n].argmax()
    print("doc: {} topic: {}\n...".format(n,
                                            topic_most_pr
                                            ))


document: 0 sum: 0.9999999999999996
document: 1 sum: 0.9999999999999998
document: 2 sum: 1.0000000000000002
document: 3 sum: 1.0
document: 4 sum: 0.9999999999999994
doc: 0 topic: 19
...
doc: 1 topic: 6
...
doc: 2 topic: 4
...
doc: 3 topic: 6
...
doc: 4 topic: 6
...
doc: 5 topic: 6
...
doc: 6 topic: 6
...
doc: 7 topic: 13
...
doc: 8 topic: 6
...
doc: 9 topic: 6
...


In [8]:
# Building JS Matrix


def multi_js(p, q):
    """Jensen-Shannon divergence (symmetric) between two multinomials,
    expressed in nats."""
    if (len(q.shape) == 2):
        axis = 1
    else:
        axis = 0
    # D_{JS}(P\|Q) = (D_{KL}(P\|Q) + D_{KL}(Q\|P)) / 2
    return 0.5 * ((q * (np.log(q.clip(1e-10,1))
                        - np.log(p.clip(1e-10,1)))).sum(axis)
                      + (p * (np.log(p.clip(1e-10,1))
                              - np.log(q.clip(1e-10,1)))).sum(axis))


JS_D1D2 = multi_js(doc_topic[0],doc_topic[1])
JS_D1D2

2.9541148438200611

In [9]:
# Alternative JS implementation

def jsd(x,y): #Jensen-shannon divergence
    import warnings
    warnings.filterwarnings("ignore", category = RuntimeWarning)
    x = np.array(x)
    y = np.array(y)
    d1 = x*np.log2(2*x/(x+y))
    d2 = y*np.log2(2*y/(x+y))
    d1[np.isnan(d1)] = 0
    d2[np.isnan(d2)] = 0
    d = 0.5*np.sum(d1+d2)    
    return d

jsd(doc_topic[0],doc_topic[1])

0.60293308491694719

In [10]:
# Constructing JS Matrix

# Creates a list containing 5 lists, each of 8 items, all set to 0
w, h = 23, 23 
Matrix = [[0 for x in range(w)] for y in range(h)] 

JSM = list()
js_df = pd.DataFrame()

# for index1,item1 in enumerate(doc_topic):
#     JSM[:] = []
#     for index2,item2 in enumerate(doc_topic):
#         result = jsd(doc_topic[index1],doc_topic[index2])
#         JSM.append(result)
#         js_df = pd.DataFrame(JSM)
#         js_df = js_df.transpose()
#     js_df1.append(js_df)
# #     js_df.loc[index1] = pd.DataFrame.append(JSM)

# print(js_df1)

# len(df.columns) = 23


# for index1,item1 in enumerate(doc_topic):
#     for index2,item2 in enumerate(doc_topic):
#         result = jsd(doc_topic[index1],doc_topic[index2])
#         js_df[index1,index2] = pd.DataFrame(result)
        
# print(js_df)


for index1,item1 in enumerate(doc_topic):
    for index2,item2 in enumerate(doc_topic):
        result = jsd(doc_topic[index1],doc_topic[index2])
        Matrix[index1][index2] = result
        temp_list = list()        
        temp_list = (index1,index2,result)
        JSM.append(temp_list)

print(JSM)

[(0, 0, 0.0), (0, 1, 0.60293308491694719), (0, 2, 0.48943516134082565), (0, 3, 0.39298711843958667), (0, 4, 0.49684648549978927), (0, 5, 0.55107657859605375), (0, 6, 0.55144752863278979), (0, 7, 0.64456330984810206), (0, 8, 0.72024317949805527), (0, 9, 0.43490975188432368), (0, 10, 0.49101141981905727), (0, 11, 0.60822074484477839), (0, 12, 0.40516684161102456), (0, 13, 0.44303672131471122), (0, 14, 0.4302340827337664), (0, 15, 0.53153753140193538), (0, 16, 0.3843143784569622), (0, 17, 0.52873183935586476), (0, 18, 0.49486887476451263), (0, 19, 0.42697360239745585), (0, 20, 0.5034051653071242), (0, 21, 0.61456366481394875), (0, 22, 0.53195399357818995), (1, 0, 0.60293308491694719), (1, 1, 0.0), (1, 2, 0.44408218792809329), (1, 3, 0.39377654056400546), (1, 4, 0.24645438108943146), (1, 5, 0.29243696046069767), (1, 6, 0.26693326533002648), (1, 7, 0.49836923270994471), (1, 8, 0.27899994084519042), (1, 9, 0.45492180175777064), (1, 10, 0.28145435588301732), (1, 11, 0.34413738276731265), (1, 

In [13]:
# Writing to a txt file for Mapequation

# JS_str = str(JSM)

text_file = open("/Users/tarunruchandani/Desktop/HarvardSummer2016/mymaps/JS_output1.txt", "w")

for index1,item1 in enumerate(doc_topic):
    for index2,item2 in enumerate(doc_topic):
        result = 1-(jsd(doc_topic[index1],doc_topic[index2]))
        text_file.write("%s %s %s\n" %(index1, index2, result))

text_file.close()

